<a href="https://colab.research.google.com/github/saritha367/MovieRecommendation-chatbot/blob/main/movie_reccomendationFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
# Load the file (replace with correct path if needed)
df = pd.read_csv('indian_movies_updated.csv')

# Check column names
print(df.columns)
df.columns = df.columns.str.strip().str.lower()

In [ ]:
# Mapping for common genre synonyms
genre_synonyms = {
    'romantic': 'romance',
    'sci fi': 'sci-fi',
    'science fiction': 'sci-fi',
    'biographical': 'biography',
    'thrillers': 'thriller',
    'action-packed': 'action',
    'musicals': 'musical'
}

def recommend_indian_movies(user_input, num_results=5):
    words = user_input.lower().split()
    user_lang = None
    user_genre = None

    known_languages = df['language'].str.lower().unique().tolist()
    all_genres = df['genres'].str.lower().str.split(', ').explode().unique().tolist()

    # Apply synonym mapping
    words = [genre_synonyms.get(word, word) for word in words]

    for word in words:
        if word in known_languages:
            user_lang = word
        if word in all_genres:
            user_genre = word

    if not user_lang or not user_genre:
        return ["❌ Please enter both a valid Indian language and genre (e.g., 'Tamil thriller')."]

    filtered = df[(df['language'].str.lower() == user_lang) & (df['genres'].str.lower().str.contains(user_genre))]

    if filtered.empty:
        return [f"❌ No {user_genre.title()} movies found in {user_lang.title()}."]

    filtered = filtered.sort_values(by='rating', ascending=False).head(num_results)
    return [f"🎬 Top {user_genre.title()} movies in {user_lang.title()}:"] + filtered['title'].tolist()


In [ ]:
# Chatbot interface
print("🎬 Welcome to the Indian Language Movie Bot!")
print("Type a genre and language (e.g., 'Tamil thriller', 'Hindi drama'):")
print("❌ Type 'exit' to quit.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Bot: 👋 Goodbye!")
        break
    recommendations = recommend_indian_movies(user_input)
    print("Bot:", recommendations[0])
    for movie in recommendations[1:]:
        print(f"  👉 {movie}")
    print()